## Deep Convolutional Generative Adversarial Network (DCGAN) Tutorial

Refer to: https://gist.github.com/awjuliani/8ebf356d03ffee139659807be7fa2611#file-dcgan-ipynb

In [1]:
#Import the libraries we will need.
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.contrib.slim as slim
import os
import scipy.misc
import scipy

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Helper Function

In [3]:
#This function performns a leaky relu activation, which is needed for the discriminator network.
def lrelu(x, leak=0.2, name="lrelu"):
     with tf.variable_scope(name):
         f1 = 0.5 * (1 + leak)
         f2 = 0.5 * (1 - leak)
         return f1 * x + f2 * abs(x)
    
#The below functions are taken from carpdem20's implementation https://github.com/carpedm20/DCGAN-tensorflow
#They allow for saving sample images from the generator to follow progress
def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)

def imsave(images, size, path):
    return scipy.misc.imsave(path, merge(images, size))

def inverse_transform(images):
    return (images+1.)/2.

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1]))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = image

    return img

## Defining the Adversarial Networks

### Generator Network

In [4]:
def generator(z):
    
    zP = slim.fully_connected(z,4*4*256,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_project',weights_initializer=initializer)
    zCon = tf.reshape(zP,[-1,4,4,256])
    
    gen1 = slim.convolution2d_transpose(\
        zCon,num_outputs=64,kernel_size=[5,5],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv1', weights_initializer=initializer)
    
    gen2 = slim.convolution2d_transpose(\
        gen1,num_outputs=32,kernel_size=[5,5],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv2', weights_initializer=initializer)
    
    gen3 = slim.convolution2d_transpose(\
        gen2,num_outputs=16,kernel_size=[5,5],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv3', weights_initializer=initializer)
    
    g_out = slim.convolution2d_transpose(\
        gen3,num_outputs=1,kernel_size=[32,32],padding="SAME",\
        biases_initializer=None,activation_fn=tf.nn.tanh,\
        scope='g_out', weights_initializer=initializer)
    
    return g_out

### Discriminator Network

In [5]:
def discriminator(bottom, reuse=False):
    
    dis1 = slim.convolution2d(bottom,16,[4,4],stride=[2,2],padding="SAME",\
        biases_initializer=None,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv1',weights_initializer=initializer)
    
    dis2 = slim.convolution2d(dis1,32,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv2', weights_initializer=initializer)
    
    dis3 = slim.convolution2d(dis2,64,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv3',weights_initializer=initializer)
    
    d_out = slim.fully_connected(slim.flatten(dis3),1,activation_fn=tf.nn.sigmoid,\
        reuse=reuse,scope='d_out', weights_initializer=initializer)
    
    return d_out

### Connecting them together

In [6]:
tf.reset_default_graph()

z_size = 100 #Size of z vector used for generator.

#This initializaer is used to initialize all the weights of the network.
initializer = tf.truncated_normal_initializer(stddev=0.02)

#These two placeholders are used for input into the generator and discriminator, respectively.
z_in = tf.placeholder(shape=[None,z_size],dtype=tf.float32) #Random vector
real_in = tf.placeholder(shape=[None,32,32,1],dtype=tf.float32) #Real images

Gz = generator(z_in) #Generates images from random z vectors
Dx = discriminator(real_in) #Produces probabilities for real images
Dg = discriminator(Gz,reuse=True) #Produces probabilities for generator images

#These functions together define the optimization objective of the GAN.
d_loss = -tf.reduce_mean(tf.log(Dx) + tf.log(1.-Dg)) #This optimizes the discriminator.
g_loss = -tf.reduce_mean(tf.log(Dg)) #This optimizes the generator.

tvars = tf.trainable_variables()

#The below code is responsible for applying gradient descent to update the GAN.
trainerD = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
trainerG = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
d_grads = trainerD.compute_gradients(d_loss,tvars[9:]) #Only update the weights for the discriminator network.
g_grads = trainerG.compute_gradients(g_loss,tvars[0:9]) #Only update the weights for the generator network.

update_D = trainerD.apply_gradients(d_grads)
update_G = trainerG.apply_gradients(g_grads)

### Training the network

In [7]:
batch_size = 128 #Size of image batch to apply at each iteration.
iterations = 500000 #Total number of iterations to use.
sample_directory = './MNIST_3_figs' #Directory to save sample images from generator in.
model_directory = './MNIST_3_models' #Directory to save trained model to.

init = tf.initialize_all_variables()
saver = tf.train.Saver()
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:  
    sess.run(init)
    for i in range(iterations):
        zs = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32) #Generate a random z batch
        xs,_ = mnist.train.next_batch(batch_size) #Draw a sample batch from MNIST dataset.
        xs = (np.reshape(xs,[batch_size,28,28,1]) - 0.5) * 2.0 #Transform it to be between -1 and 1
        xs = np.lib.pad(xs, ((0,0),(2,2),(2,2),(0,0)),'constant', constant_values=(-1, -1)) #Pad the images so the are 32x32
        _,dLoss = sess.run([update_D,d_loss],feed_dict={z_in:zs,real_in:xs}) #Update the discriminator
        _,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs}) #Update the generator, twice for good measure.
        _,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs})
        if i % 10 == 0:
            print("Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss))
            z2 = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32) #Generate another z batch
            newZ = sess.run(Gz,feed_dict={z_in:z2}) #Use new z to get sample images from generator.
            if not os.path.exists(sample_directory):
                os.makedirs(sample_directory)
            #Save sample generator images for viewing training progress.
            save_images(np.reshape(newZ[0:36],[36,32,32]),[6,6],sample_directory+'/fig'+str(i)+'.png')
        if i % 1000 == 0 and i != 0:
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            saver.save(sess,model_directory+'/model-'+str(i)+'.cptk')
            print("Saved Model")

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Gen Loss: 0.437057 Disc Loss: 1.38158
Gen Loss: 0.190678 Disc Loss: 2.46454
Gen Loss: 0.334599 Disc Loss: 2.1929
Gen Loss: 0.36985 Disc Loss: 2.02255
Gen Loss: 0.592475 Disc Loss: 1.59378
Gen Loss: 0.4271 Disc Loss: 1.9811
Gen Loss: 0.705615 Disc Loss: 1.62642
Gen Loss: 0.68682 Disc Loss: 1.55711
Gen Loss: 1.07092 Disc Loss: 1.15507
Gen Loss: 0.710663 Disc Loss: 1.65106
Gen Loss: 0.739506 Disc Loss: 1.57527
Gen Loss: 0.769349 Disc Loss: 1.23161
Gen Loss: 0.835697 Disc Loss: 1.40018
Gen Loss: 0.970331 Disc Loss: 1.19534
Gen Loss: 0.7775 Disc Loss: 1.44406
Gen Loss: 0.553028 Disc Loss: 1.77047
Gen Loss: 1.10858 Disc Loss: 1.07085
Gen Loss: 0.886057 Disc Loss: 1.47932
Gen Loss: 1.11071 Disc Loss: 1.08279
Gen Loss: 1.27632 Disc Loss: 0.957053
Gen Loss: 1.06103 Disc Loss: 1.30106
Gen Loss: 0.829228 Disc Loss: 1.20386
Gen Loss: 1.14409 Disc Loss: 0.915108
Gen Loss: 1.07147 Disc Loss: 1.34198
Gen Loss: 1.10733 Disc Loss

Gen Loss: 1.30057 Disc Loss: 0.871206
Gen Loss: 1.33708 Disc Loss: 0.914082
Gen Loss: 1.51346 Disc Loss: 0.768983
Gen Loss: 1.347 Disc Loss: 0.848753
Gen Loss: 1.33366 Disc Loss: 0.842553
Gen Loss: 1.59579 Disc Loss: 0.704741
Gen Loss: 1.15581 Disc Loss: 1.4804
Gen Loss: 1.45076 Disc Loss: 0.882807
Gen Loss: 1.85153 Disc Loss: 0.563493
Gen Loss: 1.31084 Disc Loss: 1.14448
Gen Loss: 1.33549 Disc Loss: 1.32413
Gen Loss: 1.67954 Disc Loss: 0.662537
Gen Loss: 1.8588 Disc Loss: 0.479883
Gen Loss: 1.66245 Disc Loss: 0.807073
Gen Loss: 1.82814 Disc Loss: 0.593045
Gen Loss: 1.97673 Disc Loss: 0.689022
Gen Loss: 1.24488 Disc Loss: 0.818411
Gen Loss: 1.75996 Disc Loss: 0.677894
Gen Loss: 1.48961 Disc Loss: 1.18638
Gen Loss: 1.32413 Disc Loss: 0.779561
Gen Loss: 1.54903 Disc Loss: 0.753681
Gen Loss: 1.47109 Disc Loss: 0.657784
Gen Loss: 1.38859 Disc Loss: 0.872723
Gen Loss: 1.93892 Disc Loss: 1.05729
Gen Loss: 1.58451 Disc Loss: 0.725939
Gen Loss: 1.37904 Disc Loss: 0.687172
Gen Loss: 1.5038 Disc

Gen Loss: 1.69652 Disc Loss: 0.494805
Gen Loss: 1.97198 Disc Loss: 0.575561
Gen Loss: 2.17341 Disc Loss: 0.896843
Gen Loss: 2.11691 Disc Loss: 0.593725
Gen Loss: 1.56825 Disc Loss: 0.710781
Gen Loss: 2.01433 Disc Loss: 0.564706
Gen Loss: 2.9244 Disc Loss: 1.06706
Gen Loss: 2.54186 Disc Loss: 0.337927
Gen Loss: 1.62733 Disc Loss: 0.549834
Gen Loss: 1.81615 Disc Loss: 0.609823
Gen Loss: 1.77886 Disc Loss: 0.558429
Gen Loss: 1.03807 Disc Loss: 0.952887
Gen Loss: 1.50839 Disc Loss: 0.669505
Gen Loss: 1.73113 Disc Loss: 0.575639
Gen Loss: 1.31037 Disc Loss: 0.775476
Gen Loss: 1.54974 Disc Loss: 0.788924
Gen Loss: 1.89694 Disc Loss: 0.513657
Gen Loss: 1.5945 Disc Loss: 0.603069
Gen Loss: 1.66614 Disc Loss: 0.618201
Gen Loss: 1.7452 Disc Loss: 0.630896
Gen Loss: 1.72805 Disc Loss: 0.814827
Gen Loss: 1.47361 Disc Loss: 0.640314
Gen Loss: 1.87094 Disc Loss: 0.595774
Gen Loss: 2.00311 Disc Loss: 0.534368
Gen Loss: 1.63453 Disc Loss: 0.525683
Gen Loss: 1.8557 Disc Loss: 0.575856
Gen Loss: 1.8807 

Gen Loss: 2.46191 Disc Loss: 0.365869
Gen Loss: 2.35436 Disc Loss: 0.603975
Gen Loss: 1.85628 Disc Loss: 0.911006
Gen Loss: 1.55991 Disc Loss: 0.560199
Gen Loss: 1.45383 Disc Loss: 0.626027
Gen Loss: 2.33899 Disc Loss: 0.963997
Gen Loss: 2.46685 Disc Loss: 0.544449
Gen Loss: 1.83227 Disc Loss: 0.550961
Gen Loss: 1.67079 Disc Loss: 0.754623
Gen Loss: 2.13065 Disc Loss: 0.517245
Gen Loss: 1.66379 Disc Loss: 0.432714
Gen Loss: 2.00215 Disc Loss: 0.361069
Gen Loss: 1.76664 Disc Loss: 0.479071
Gen Loss: 2.20444 Disc Loss: 0.71186
Gen Loss: 2.48778 Disc Loss: 0.521662
Gen Loss: 2.07027 Disc Loss: 0.402084
Gen Loss: 2.35641 Disc Loss: 0.384427
Gen Loss: 2.13541 Disc Loss: 0.677103
Gen Loss: 1.64259 Disc Loss: 0.684737
Gen Loss: 2.10486 Disc Loss: 0.752094
Gen Loss: 2.05217 Disc Loss: 0.713244
Gen Loss: 1.66649 Disc Loss: 0.495133
Gen Loss: 1.62042 Disc Loss: 0.585769
Gen Loss: 1.74298 Disc Loss: 0.690779
Gen Loss: 1.75436 Disc Loss: 0.565588
Gen Loss: 2.60561 Disc Loss: 0.44574
Gen Loss: 1.46

Gen Loss: 2.00276 Disc Loss: 0.463265
Gen Loss: 2.92876 Disc Loss: 0.533399
Gen Loss: 2.2071 Disc Loss: 0.629879
Gen Loss: 1.89841 Disc Loss: 0.288161
Gen Loss: 2.49041 Disc Loss: 0.518811
Gen Loss: 1.71554 Disc Loss: 0.326874
Gen Loss: 2.06681 Disc Loss: 0.474932
Gen Loss: 2.55609 Disc Loss: 0.353449
Gen Loss: 0.854012 Disc Loss: 0.673013
Gen Loss: 2.96403 Disc Loss: 0.30834
Gen Loss: 1.57328 Disc Loss: 0.548591
Gen Loss: 2.25685 Disc Loss: 0.265467
Gen Loss: 2.00791 Disc Loss: 0.518598
Gen Loss: 2.38383 Disc Loss: 0.315115
Gen Loss: 1.67762 Disc Loss: 0.604958
Gen Loss: 2.14315 Disc Loss: 0.677901
Gen Loss: 2.69052 Disc Loss: 0.550779
Gen Loss: 2.29378 Disc Loss: 0.501356
Gen Loss: 2.32901 Disc Loss: 0.379098
Gen Loss: 2.87021 Disc Loss: 0.271096
Gen Loss: 3.4503 Disc Loss: 0.223319
Gen Loss: 2.47087 Disc Loss: 0.270805
Gen Loss: 1.99749 Disc Loss: 0.382572
Gen Loss: 2.06217 Disc Loss: 0.42302
Gen Loss: 2.18559 Disc Loss: 0.691166
Gen Loss: 1.31015 Disc Loss: 0.769197
Gen Loss: 0.682

...

Gen Loss: 1.91274 Disc Loss: 0.293697
Gen Loss: 2.13364 Disc Loss: 0.615142
Gen Loss: 2.17637 Disc Loss: 0.606081
Gen Loss: 1.25359 Disc Loss: 0.43597
Gen Loss: 2.00301 Disc Loss: 0.647525
Gen Loss: 2.11043 Disc Loss: 0.340978
Gen Loss: 0.927251 Disc Loss: 0.797767
Gen Loss: 1.35009 Disc Loss: 0.808285
Gen Loss: 0.721566 Disc Loss: 0.714804
Gen Loss: 2.53894 Disc Loss: 0.257958
Gen Loss: 2.04475 Disc Loss: 0.338982
Gen Loss: 1.27322 Disc Loss: 0.704186
Gen Loss: 1.80051 Disc Loss: 0.784192
Gen Loss: 1.66332 Disc Loss: 0.597989
Gen Loss: 0.961414 Disc Loss: 0.673344
Gen Loss: 2.40114 Disc Loss: 1.30825
Gen Loss: 1.19658 Disc Loss: 0.757415
Gen Loss: 0.389074 Disc Loss: 1.1114
Gen Loss: 1.34643 Disc Loss: 0.675466
Gen Loss: 1.79199 Disc Loss: 0.813583
Gen Loss: 1.09567 Disc Loss: 0.485321
Gen Loss: 2.20437 Disc Loss: 0.248344
Gen Loss: 1.81414 Disc Loss: 0.46771
Gen Loss: 1.59539 Disc Loss: 0.451331
Gen Loss: 1.79884 Disc Loss: 0.40894
Gen Loss: 2.48137 Disc Loss: 0.368785
Gen Loss: 1.19

Gen Loss: 1.87334 Disc Loss: 0.303005
Gen Loss: 1.84901 Disc Loss: 0.696774
Gen Loss: 1.87731 Disc Loss: 0.488162
Gen Loss: 1.33331 Disc Loss: 0.602021
Gen Loss: 2.47694 Disc Loss: 0.632408
Gen Loss: 1.3053 Disc Loss: 0.297499
Gen Loss: 1.45811 Disc Loss: 0.518221
Gen Loss: 1.26477 Disc Loss: 0.661342
Gen Loss: 2.77392 Disc Loss: 0.270737
Gen Loss: 1.16905 Disc Loss: 0.742931
Gen Loss: 2.75853 Disc Loss: 0.552821
Gen Loss: 0.317959 Disc Loss: 1.10591
Gen Loss: 1.9655 Disc Loss: 0.504856
Gen Loss: 1.79933 Disc Loss: 0.972121
Gen Loss: 2.15012 Disc Loss: 0.317949
Gen Loss: 1.82186 Disc Loss: 0.95928
Gen Loss: 2.07281 Disc Loss: 0.248008
Gen Loss: 1.46901 Disc Loss: 0.317472
Gen Loss: 0.693982 Disc Loss: 0.81145
Gen Loss: 1.45855 Disc Loss: 0.273753
Gen Loss: 1.47911 Disc Loss: 0.3377
Gen Loss: 2.56992 Disc Loss: 0.60235
Gen Loss: 1.65961 Disc Loss: 1.45036
Gen Loss: 1.29489 Disc Loss: 0.807518
Gen Loss: 1.89479 Disc Loss: 0.492009
Gen Loss: 0.979384 Disc Loss: 0.529602
Gen Loss: 2.05967 

Gen Loss: 1.69821 Disc Loss: 0.404988
Gen Loss: 2.12349 Disc Loss: 0.673717
Gen Loss: 2.76901 Disc Loss: 0.239505
Gen Loss: 1.89698 Disc Loss: 0.404401
Gen Loss: 2.52613 Disc Loss: 0.135329
Gen Loss: 2.56057 Disc Loss: 0.571242
Gen Loss: 2.38845 Disc Loss: 0.315604
Gen Loss: 2.41725 Disc Loss: 0.118235
Gen Loss: 1.65823 Disc Loss: 0.773073
Gen Loss: 1.34057 Disc Loss: 0.320063
Gen Loss: 3.42536 Disc Loss: 0.141879
Gen Loss: 2.82701 Disc Loss: 0.163955
Gen Loss: 1.22631 Disc Loss: 0.335612
Gen Loss: 1.7421 Disc Loss: 0.26627
Gen Loss: 1.96699 Disc Loss: 0.252544
Gen Loss: 2.43618 Disc Loss: 0.299222
Gen Loss: 1.95924 Disc Loss: 0.388419
Gen Loss: 2.80649 Disc Loss: 0.141578
Gen Loss: 2.3041 Disc Loss: 0.410844
Gen Loss: 2.40011 Disc Loss: 0.137832
Gen Loss: 2.68004 Disc Loss: 0.268861
Gen Loss: 2.1282 Disc Loss: 0.279127
Gen Loss: 1.5019 Disc Loss: 0.386494
Gen Loss: 2.13123 Disc Loss: 0.59022
Gen Loss: 1.2833 Disc Loss: 0.667433
Gen Loss: 1.93928 Disc Loss: 0.480732
Gen Loss: 1.5955 Di

Gen Loss: 2.18352 Disc Loss: 0.467957
Gen Loss: 1.60999 Disc Loss: 0.617162
Gen Loss: 1.52203 Disc Loss: 0.714602
Gen Loss: 1.29297 Disc Loss: 0.345939
Gen Loss: 2.10839 Disc Loss: 0.134129
Gen Loss: 1.64398 Disc Loss: 0.513975
Gen Loss: 1.61768 Disc Loss: 0.651741
Gen Loss: 2.1324 Disc Loss: 0.316676
Gen Loss: 2.01727 Disc Loss: 0.452751
Gen Loss: 0.537074 Disc Loss: 0.582859
Gen Loss: 1.6889 Disc Loss: 0.38704
Gen Loss: 1.90199 Disc Loss: 0.736
Gen Loss: 1.55738 Disc Loss: 0.536242
Gen Loss: 2.3123 Disc Loss: 0.47621
Gen Loss: 0.764497 Disc Loss: 0.838921
Gen Loss: 1.1331 Disc Loss: 0.38478
Gen Loss: 1.12928 Disc Loss: 0.765222
Gen Loss: 2.65227 Disc Loss: 0.279859
Gen Loss: 1.52607 Disc Loss: 0.575093
Gen Loss: 1.2049 Disc Loss: 0.565597
Gen Loss: 1.79611 Disc Loss: 0.53873
Gen Loss: 1.45465 Disc Loss: 0.432991
Gen Loss: 1.37579 Disc Loss: 0.460108
Gen Loss: 2.44046 Disc Loss: 0.254006
Gen Loss: 1.49899 Disc Loss: 0.833336
Gen Loss: 0.781214 Disc Loss: 0.6555
Gen Loss: 1.91768 Disc 

Gen Loss: 1.98522 Disc Loss: 0.350056
Gen Loss: 2.61604 Disc Loss: 0.344138
Gen Loss: 2.69443 Disc Loss: 0.459325
Gen Loss: 1.59611 Disc Loss: 0.313912
Gen Loss: 2.45804 Disc Loss: 0.34518
Gen Loss: 1.28557 Disc Loss: 0.423621
Gen Loss: 3.13203 Disc Loss: 0.0842519
Gen Loss: 2.18443 Disc Loss: 0.439885
Gen Loss: 2.56297 Disc Loss: 0.292539


### Using a trained network

In [11]:
sample_directory = './MNIST_3_figs' #Directory to save sample images from generator in.
model_directory = './MNIST_3_models' #Directory to load trained model from.
batch_size_sample = 36

init = tf.initialize_all_variables()
saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(init)
    #Reload the model.
    print('Loading Model...')
    ckpt = tf.train.get_checkpoint_state(model_directory)
    saver.restore(sess,ckpt.model_checkpoint_path)
    
    zs = np.random.uniform(-1.0,1.0,size=[batch_size_sample,z_size]).astype(np.float32) #Generate a random z batch
    newZ = sess.run(Gz,feed_dict={z_in:zs}) #Use new z to get sample images from generator.
    if not os.path.exists(sample_directory):
        os.makedirs(sample_directory)
    save_images(np.reshape(newZ[0:batch_size_sample],[36,32,32]),[6,6],sample_directory+'/fig'+str(i)+'.png')
    print(sample_directory+'/fig'+str(i)+'.png')

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Loading Model...
INFO:tensorflow:Restoring parameters from ./MNIST_3_models/model-499000.cptk
./MNIST_3_figs/fig499999.png
